## FIZZ-21 Generate visualizations with the data from canvas

Basic configuration and import csv files

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
os.chdir(os.getenv("ANALIZE_CSV_INPUT_PATH"))

assignments_df = pd.read_csv('assignments.csv')
courses_df = pd.read_csv('courses.csv')
enrollments_df = pd.read_csv('enrollments.csv')
students_df = pd.read_csv('students.csv')
submissions_df = pd.read_csv('submissions.csv')

## Clean unnecessary rows

In [2]:
submissions_df = submissions_df[["id", "score", "assignment_id", "user_id", "late", "missing", "preview_url"]]

students_df = students_df[["id", "sortable_name", "email"]]

enrollments_df = enrollments_df[["user_id", "course_id", "type"]]

courses_df = courses_df[["id", "name", "course_code"]]

assignments_df = assignments_df[["id", "description", "due_at", "unlock_at", "lock_at", "course_id", "name"]]

## Join rows by id

In [3]:
student_assignments = pd.merge(enrollments_df, assignments_df, on="course_id", how="inner")
student_assignments.drop(["due_at", "unlock_at", "lock_at", "description", "name", "type"], axis=1, inplace=True)
student_assignments.rename({'id': 'assignment_id'}, axis=1, inplace=True)

student_assignments.head()

,user_id,course_id,assignment_id
0,13,14,7
1,13,14,3
2,18,14,7
3,18,14,3
4,19,14,7


## Get students with missing assignments' information

In [4]:
student_assignments_submissions = pd.merge(student_assignments, submissions_df, on=["user_id", "assignment_id"], how="left")
student_assignments_submissions.drop(["preview_url","late","missing","score"], axis=1, inplace= True)
student_assignments_submissions.rename({'id': 'submission_id'}, axis=1, inplace=True)
student_assignments_submissions = student_assignments_submissions[student_assignments_submissions['submission_id'].isnull()]

student_with_missing_submissions = pd.merge(
    student_assignments_submissions, 
    students_df.rename({'id': 'user_id'}, axis=1),
    on="user_id", how="left")

student_with_missing_submissions = pd.merge(
    student_with_missing_submissions,
    assignments_df.rename({'id': 'assignment_id'}, axis=1),
    on="assignment_id", how="left")

student_with_missing_submissions.rename({'course_id_y': 'course_id'}, axis=1, inplace=True)

student_with_missing_submissions = pd.merge(
    student_with_missing_submissions,
    courses_df.rename({'id': 'course_id'}, axis=1),
    on="course_id", how="left")


student_with_missing_submissions.drop(
    ["course_id_x", "due_at", "unlock_at", "lock_at", "description", "description"],
    axis=1, inplace=True)

student_with_missing_submissions.rename({'name_x': 'assignment_name', 'name_y': 'course_name'}, axis=1, inplace=True)

output_path = os.getenv("ANALIZE_CSV_OUTPUT_PATH")+"\students_with_missing_submissions.csv"

student_with_missing_submissions.to_csv(output_path,index=False)
student_with_missing_submissions.head(20)

,user_id,assignment_id,submission_id,sortable_name,email,course_id,assignment_name,course_name,course_code
0,19,7,NaN,"Hardy, Olivia Doris",Olivia.Hardy@studentgps.org,14,ENG Homework 1,English I,ENG-1
1,20,7,NaN,"Turner, Micheal",Micheal.Turner@studentgps.org,14,ENG Homework 1,English I,ENG-1
2,20,3,NaN,"Turner, Micheal",Micheal.Turner@studentgps.org,14,ENG Reading 1,English I,ENG-1
3,14,5,NaN,"Hughes, Kyle",Kyle.Hughes@studentgps.org,16,ALG Homework 1,Algebra I,03100500
4,16,6,NaN,"Mahoney, Larry",Larry.Mahoney@studentgps.org,16,ALG Homework 2,Algebra I,03100500
5,15,6,NaN,"Nash, Peter Ivan",Peter.Nash@studentgps.org,16,ALG Homework 2,Algebra I,03100500
6,17,5,NaN,"Phillips, Roland",Roland.Phillips@studentgps.org,16,ALG Homework 1,Algebra I,03100500
